 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8889/notebooks/json_preprocess.ipynb#预处理一" data-toc-modified-id="预处理一-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>预处理一</a></span></li></ul></div>

In [ ]:
from __future__ import print_function
import os
import argparse
import json
from PIL import Image
import jieba

In [2]:
train_caption_json = '/home/jiangqy/EXP/competition/data/captions/ai_challenger_caption_train_20170902/caption_train_annotations_20170902.json'
train_img_dir = '/home/jiangqy/EXP/competition/data/captions/ai_challenger_caption_train_20170902/caption_train_images_20170902'
val_caption_json = '/home/jiangqy/EXP/competition/data/captions/ai_challenger_caption_validation_20170910/caption_validation_annotations_20170910.json'
val_img_dir = '/home/jiangqy/EXP/competition/data/captions/ai_challenger_caption_validation_20170910/caption_validation_images_20170910'
test_img_dir = '/home/jiangqy/EXP/competition/data/captions/ai_challenger_caption_test1_20170923/caption_test1_images_20170923'

In [29]:
# change original form into coco form 
# save the changed form in out_put_file

def convert2coco(caption_json, img_dir):
    dataset = json.load(open(caption_json, 'r'))
    imgdir = img_dir

    coco = dict()
    coco[u'info'] = { u'desciption':u'AI challenger image caption in mscoco format'}
    coco[u'licenses'] = ['Unknown', 'Unknown']
    coco[u'images'] = list()
    coco[u'annotations'] = list()

    for ind, sample in enumerate(dataset):
        img = Image.open(os.path.join(imgdir, sample['image_id']))
        width, height = img.size

        coco_img = {}
        coco_img[u'license'] = 0
        coco_img[u'file_name'] = os.path.split(img_dir)[-1]+'/'+sample['image_id']
        coco_img[u'width'] = width
        coco_img[u'height'] = height
        coco_img[u'date_captured'] = 0
        coco_img[u'coco_url'] = sample['url']
        coco_img[u'flickr_url'] = sample['url']
        coco_img['id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]

        coco_anno = {}
        coco_anno[u'image_id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]
        coco_anno[u'id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]
        coco_anno[u'caption'] = sample['caption']

        coco[u'images'].append(coco_img)
        coco[u'annotations'].append(coco_anno)
        if ind % 1000 == 0:
            print('{}/{}'.format(ind, len(dataset)))

    output_file = os.path.join('./json_preprocess_data', 'coco_'+os.path.basename(caption_json))
    with open(output_file, 'w') as fid:
        json.dump(coco, fid)
    print('Saved to {}'.format(output_file))
    return coco

In [30]:
# The same as the function above,
# difference is that it adds a function to judge if there is some caption without words. then fill in it with the above words.
def convert2coco_val(caption_json, img_dir):
    dataset = json.load(open(caption_json, 'r'))
    imgdir = img_dir

    coco = dict()
    coco[u'info'] = { u'desciption':u'AI challenger image caption in mscoco format'}
    coco[u'licenses'] = ['Unknown', 'Unknown']
    coco[u'images'] = list()
    coco[u'annotations'] = list()

    for ind, sample in enumerate(dataset):
        img = Image.open(os.path.join(imgdir, sample['image_id']))
        width, height = img.size

        coco_img = {}
        coco_img[u'license'] = 0
        coco_img[u'file_name'] = os.path.split(img_dir)[-1]+'/'+sample['image_id']
        coco_img[u'width'] = width
        coco_img[u'height'] = height
        coco_img[u'date_captured'] = 0
        coco_img[u'coco_url'] = sample['url']
        coco_img[u'flickr_url'] = sample['url']
        coco_img['id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]

        coco_anno = {}
        coco_anno[u'image_id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]
        coco_anno[u'id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]
        coco_anno[u'caption'] = sample['caption']
        idx = 0
        for s in sample['caption']: 
            if len(s)==0:
                print(ind,'***********')
                print('error: some caption had no words?')
                print(coco_img[u'file_name'])
                sample['caption'][idx] = sample['caption'][idx-1]
                print(sample['caption'])
                print('haha',len(sample['caption']),len(coco_anno[u'caption']))
                #break
            idx = idx+1
        coco[u'images'].append(coco_img)
        coco[u'annotations'].append(coco_anno)
        if ind % 1000 == 0:
            print('{}/{}'.format(ind, len(dataset)))

    output_file = os.path.join('./json_preprocess_data', 'coco_'+os.path.basename(caption_json))
    with open(output_file, 'w') as fid:
        json.dump(coco, fid)
    print('Saved to {}'.format(output_file))
    return coco

In [31]:
def convert2coco_eval(caption_json, img_dir):
    dataset = json.load(open(caption_json, 'r'))
    imgdir = img_dir

    coco = dict()
    coco[u'info'] = { u'desciption':u'AI challenger image caption in mscoco format'}
    coco[u'licenses'] = ['Unknown', 'Unknown']
    coco[u'images'] = list()
    coco[u'annotations'] = list()
    coco[u'type'] = u'captions'
    for ind, sample in enumerate(dataset):
        #img = Image.open(os.path.join(imgdir, sample['image_id']))
        #width, height = img.size
        width, height = 224, 224

        coco_img = {}
        coco_img[u'license'] = 0
        coco_img[u'file_name'] = os.path.split(img_dir)[-1]+'/'+sample['image_id']
        coco_img[u'width'] = width
        coco_img[u'height'] = height
        coco_img[u'date_captured'] = 0
        coco_img[u'coco_url'] = sample['url']
        coco_img[u'flickr_url'] = sample['url']
        coco_img['id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]

        coco_anno = {}
        coco_anno[u'image_id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]
        coco_anno[u'id'] = os.path.splitext(os.path.basename(sample['image_id']))[0]
        coco_anno[u'caption'] = sample['caption']

        coco[u'images'].append(coco_img)
        
        for coco_anno_ in coco_anno['caption']:
            coco_anno_s = {}
            coco_anno_s[u'image_id'] = coco_anno[u'image_id']
            coco_anno_s[u'id'] = coco_anno[u'id']
            w = jieba.cut(coco_anno_.strip(), cut_all=False)
            p = ' '.join(w)
            coco_anno_ = p
            coco_anno_s[u'caption'] = coco_anno_
            coco[u'annotations'].append(coco_anno_s)
        if ind % 1000 == 0:
            print('{}/{}'.format(ind, len(dataset)))

    output_file = os.path.join('./json_preprocess_data', 'coco_val_'+os.path.basename(caption_json))
    with open(output_file, 'w') as fid:
        json.dump(coco, fid)
    print('Saved to {}'.format(output_file))
    return coco

def convert2coco_test(img_dir):
    coco = dict()
    coco[u'info'] = { u'desciption':u'AI challenger image caption in mscoco format'}
    coco[u'licenses'] = ['Unknown', 'Unknown']
    coco[u'images'] = list()
    ind = 0
    for im_name in enumerate(os.listdir(img_dir)):
        width, height = 224, 224

        coco_img = {}
        coco_img[u'license'] = 0
        coco_img[u'file_name'] = im_name[1]
        coco_img[u'width'] = width
        coco_img[u'height'] = height
        coco_img[u'date_captured'] = 0
        coco_img[u'id'] = os.path.splitext(os.path.basename(im_name[1]))[0]
        ind = ind + 1
        coco[u'images'].append(coco_img)

        print('{}/{}'.format(ind, len(os.listdir(img_dir))))

    output_file = os.path.join('./json_preprocess_data', 'ai_challenger_test1.json')
    with open(output_file, 'w') as fid:
        json.dump(coco, fid)
    print('Saved to {}'.format(output_file)) 
    return coco

In [32]:
convert2coco(train_caption_json, train_img_dir)
convert2coco_val(val_caption_json, val_img_dir)
# Create json file for testing
convert2coco_test(test_img_dir)
# Create json file for evaluation
convert2coco_eval(val_caption_json, val_img_dir)
   

0/210000
1000/210000
2000/210000
3000/210000
4000/210000
5000/210000
6000/210000
7000/210000
8000/210000
9000/210000
10000/210000
11000/210000
12000/210000
13000/210000
14000/210000
15000/210000
16000/210000
17000/210000
18000/210000
19000/210000
20000/210000
21000/210000
22000/210000
23000/210000
24000/210000
25000/210000
26000/210000
27000/210000
28000/210000
29000/210000
30000/210000
31000/210000
32000/210000
33000/210000
34000/210000
35000/210000
36000/210000
37000/210000
38000/210000
39000/210000
40000/210000
41000/210000
42000/210000
43000/210000
44000/210000
45000/210000
46000/210000
47000/210000
48000/210000
49000/210000
50000/210000
51000/210000
52000/210000
53000/210000
54000/210000
55000/210000
56000/210000
57000/210000
58000/210000
59000/210000
60000/210000
61000/210000
62000/210000
63000/210000
64000/210000
65000/210000
66000/210000
67000/210000
68000/210000
69000/210000
70000/210000
71000/210000
72000/210000
73000/210000
74000/210000
75000/210000
76000/210000
77000/210000

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.275 seconds.
Prefix dict has been built succesfully.


0/30000
1000/30000
2000/30000
3000/30000
4000/30000
5000/30000
6000/30000
7000/30000
8000/30000
9000/30000
10000/30000
11000/30000
12000/30000
13000/30000
14000/30000
15000/30000
16000/30000
17000/30000
18000/30000
19000/30000
20000/30000
21000/30000
22000/30000
23000/30000
24000/30000
25000/30000
26000/30000
27000/30000
28000/30000
29000/30000
Saved to ./json_preprocess_data/coco_val_caption_validation_annotations_20170910.json


{u'annotations': [{u'caption': u'\u4e00\u4e2a \u53cc\u81c2 \u62ac\u8d77 \u7684 \u8fd0\u52a8\u5458 \u8dea \u5728 \u7eff\u8335\u8335 \u7684 \u7403\u573a\u4e0a',
   u'id': u'3cd32bef87ed98572bac868418521852ac3f6a70',
   u'image_id': u'3cd32bef87ed98572bac868418521852ac3f6a70'},
  {u'caption': u'\u4e00\u4e2a \u62ac\u7740 \u53cc\u81c2 \u7684 \u8fd0\u52a8\u5458 \u8dea \u5728 \u8db3\u7403\u573a \u4e0a',
   u'id': u'3cd32bef87ed98572bac868418521852ac3f6a70',
   u'image_id': u'3cd32bef87ed98572bac868418521852ac3f6a70'},
  {u'caption': u'\u4e00\u4e2a \u53cc\u624b \u63e1\u62f3 \u7684 \u7537\u4eba \u8dea \u5728 \u7eff\u8335\u8335 \u7684 \u8db3\u7403\u573a \u4e0a',
   u'id': u'3cd32bef87ed98572bac868418521852ac3f6a70',
   u'image_id': u'3cd32bef87ed98572bac868418521852ac3f6a70'},
  {u'caption': u'\u4e00\u4e2a \u62ac\u8d77 \u53cc\u624b \u7684 \u7537\u4eba \u8dea \u5728 \u78a7\u7eff \u7684 \u7403\u573a\u4e0a',
   u'id': u'3cd32bef87ed98572bac868418521852ac3f6a70',
   u'image_id': u'3cd32bef87ed98572b

In [14]:
import json
train = json.load(open('./json_preprocess_data/coco_caption_train_annotations_20170902.json', 'r'))
val = json.load(open('./json_preprocess_data/coco_caption_validation_annotations_20170910.json', 'r'))
print(val.keys())
print(val['info'])
print(len(val['images']))
print(len(val['annotations']))
print(val['images'][0])
print(val['annotations'][0])

[u'info', u'licenses', u'images', u'annotations']
{u'desciption': u'AI challenger image caption in mscoco format'}
30000
30000
{u'license': 0, u'file_name': u'caption_validation_images_20170910/3cd32bef87ed98572bac868418521852ac3f6a70.jpg', u'coco_url': u'http://img5.cache.netease.com/photo/0005/2013-09-25/99LA1FC60B6P0005.jpg', u'height': 607, u'width': 1000, u'date_captured': 0, u'flickr_url': u'http://img5.cache.netease.com/photo/0005/2013-09-25/99LA1FC60B6P0005.jpg', u'id': u'3cd32bef87ed98572bac868418521852ac3f6a70'}
{u'image_id': u'3cd32bef87ed98572bac868418521852ac3f6a70', u'id': u'3cd32bef87ed98572bac868418521852ac3f6a70', u'caption': [u'\u4e00\u4e2a\u53cc\u81c2\u62ac\u8d77\u7684\u8fd0\u52a8\u5458\u8dea\u5728\u7eff\u8335\u8335\u7684\u7403\u573a\u4e0a', u'\u4e00\u4e2a\u62ac\u7740\u53cc\u81c2\u7684\u8fd0\u52a8\u5458\u8dea\u5728\u8db3\u7403\u573a\u4e0a', u'\u4e00\u4e2a\u53cc\u624b\u63e1\u62f3\u7684\u7537\u4eba\u8dea\u5728\u7eff\u8335\u8335\u7684\u8db3\u7403\u573a\u4e0a', u'\u4e00\

In [25]:
imgs = train['images']+val['images']
annots = train['annotations']+val['annotations']

In [21]:
itoa = {}
for a in annots:
    imgid = a['image_id']
    if not imgid in itoa: itoa[imgid] = []
    itoa[imgid].append(a)

In [33]:
(itoa[u'8f00f3d0f1008e085ab660e70dffced16a8259f6'])[0]['caption']

[u'\u4e24\u4e2a\u8863\u7740\u4f11\u95f2\u7684\u4eba\u5728\u5e73\u6574\u7684\u9053\u8def\u4e0a\u4ea4\u8c08',
 u'\u4e00\u4e2a\u7a7f\u7740\u7ea2\u8272\u4e0a\u8863\u7684\u7537\u4eba\u548c\u4e00\u4e2a\u7a7f\u7740\u7070\u8272\u88e4\u5b50\u7684\u7537\u4eba\u7ad9\u5728\u5ba4\u5916\u7684\u9053\u8def\u4e0a\u4ea4\u8c08',
 u'\u5ba4\u5916\u7684\u516c\u56ed\u91cc\u6709\u4e24\u4e2a\u7a7f\u7740\u957f\u88e4\u7684\u7537\u4eba\u5728\u4ea4\u6d41',
 u'\u8857\u9053\u4e0a\u6709\u4e00\u4e2a\u7a7f\u7740\u6df1\u8272\u5916\u5957\u7684\u7537\u4eba\u548c\u4e00\u4e2a\u7a7f\u7740\u7ea2\u8272\u5916\u5957\u7684\u7537\u4eba\u5728\u4ea4\u8c08',
 u'\u9053\u8def\u4e0a\u6709\u4e00\u4e2a\u8eab\u7a7f\u7ea2\u8272\u4e0a\u8863\u7684\u7537\u4eba\u5728\u548c\u4e00\u4e2a\u62ac\u7740\u5de6\u624b\u7684\u4eba\u8bb2\u8bdd']

In [55]:
out_json={}
out=[]
import os
import jieba
for i, img in enumerate(imgs):
    out_im = {}
    # coco specific here, they store train/val images separately
    split = 'train' if 'train' in img['file_name'] else 'val'
    annotsi = itoa[img['id']]
    sentid = 0
    out_im['cocoid'] = img['id']
    out_im['filename'] = os.path.basename(img['file_name'])
    if 'val' in img['file_name']:
        out_im['filepath'] = 'ai_challenger_caption_validation_20170910/caption_validation_images_20170910'
    else:
        out_im['filepath'] = 'ai_challenger_caption_train_20170902/caption_train_images_20170902'
    out_s = [] 
    for a in annotsi: 
        txt = []
        jimg = {}
        jimg['imgid'] = img['id']
        jimg['raw'] = s
        jimg['sentid'] = img['id']+'_'+str(sentid) 
        for s in a['caption']:
            txt.append(list(jieba.cut(s)))  
        jimg['tokens'] = txt
        jimg['sentids'] = []
        out_s.append(jimg) 
    out_im['sentences'] = out_s
    out_im['split'] = split
    out.append(out_im)
out_json['images']=out
out_json['dataset']='ai_challenger'
output_file = os.path.join('./json_preprocess_data', 'coco_ai_challenger_version1.json')
json.dump(out_json, open(output_file, 'w'))

In [81]:
import cPickle
# b = cPickle.load(open('/home/jiangqy/PycharmProjects/image_Caption_copy/jupyter/log_st/infos_.pkl','r'))
b = cPickle.load(open('/home/jiangqy/PycharmProjects/Image_Caption/log_st/infos_st.pkl','r'))

In [86]:
c = cPickle.load(open('/home/jiangqy/PycharmProjects/image_Caption_copy/jupyter/log_st/infos_-best.pkl','r'))

AttributeError: 'module' object has no attribute 'Opt'

In [58]:
a = h5py.File('./json_preprocess_data/coco_ai_challenger_talk_resnet151_label.h5')

In [ ]:
# ''' 
# version 2 : put five short sentences into one long sentence
# '''
# out_json={}
# out=[]
# import os
# import jieba
# for i, img in enumerate(imgs):
#     out_im = {}
#     # coco specific here, they store train/val images separately
#     split = 'train' if 'train' in img['file_name'] else 'val'
#     annotsi = itoa[img['id']]
#     sentid = 0
#     out_im['cocoid'] = img['id']
#     out_im['filename'] = os.path.basename(img['file_name'])
#     if 'val' in img['file_name']:
#         out_im['filepath'] = 'ai_challenger_caption_validation_20170910/caption_validation_images_20170910'
#     else:
#         out_im['filepath'] = 'ai_challenger_caption_train_20170902/caption_train_images_20170902'
#     out_s = [] 
#     for a in annotsi: 
#         for s in a['caption']:
#             txt = []
#             jimg = {}
#             jimg['imgid'] = img['id']
#             jimg['raw'] = s
#             jimg['sentid'] = img['id']+'_'+str(sentid)
            
# #             for sentence in s:
# #                 print(sentence)  
# #             txt.append("".join(jieba.cut(sentence)))

#             txt.append(list(jieba.cut(s)))  
#             jimg['tokens'] = txt
#             jimg['sentids'] = []
#             out_s.append(jimg) 
#     out_im['sentences'] = out_s
#     out_im['split'] = split
#     out.append(out_im)
# out_json['images']=out
# out_json['dataset']='ai_challenger'
# output_file = os.path.join('./json_preprocess_data', 'coco_ai_challenger.json')
# json.dump(out_json, open(output_file, 'w'))

In [51]:
def ai_challenger_preprocess():
    
    train = json.load(open('./json_preprocess_data/coco_caption_train_annotations_20170902.json', 'r'))
    val = json.load(open('./json_preprocess_data/coco_caption_validation_annotations_20170910.json', 'r'))
    print(val.keys())
    print(val['info'])
    print(len(val['images']))
    print(len(val['annotations']))
    print(val['images'][0])
    print(val['annotations'][0])

    import json
    import os

    # combine all images and annotations together
    imgs = train['images']+val['images']
    annots = train['annotations']+val['annotations']

    # for efficiency lets group annotations by image
    itoa = {}
    for a in annots:
        imgid = a['image_id']
        if not imgid in itoa: itoa[imgid] = []
        itoa[imgid].append(a)

    # create the json blob
    out_json={}
    out=[]
    for i, img in enumerate(imgs):
        out_im = {}
        # coco specific here, they store train/val images separately
        split = 'train' if 'train' in img['file_name'] else 'val'
        annotsi = itoa[img['id']]
        sentid = 0
        out_im['cocoid'] = img['id']
        out_im['filename'] = os.path.basename(img['file_name'])
        if 'val' in img['file_name']:
            out_im['filepath'] = 'ai_challenger_caption_validation_20170910/caption_validation_images_20170910'
        else:
            out_im['filepath'] = 'ai_challenger_caption_train_20170902/caption_train_images_20170902'
        out_s = [] 
        for a in annotsi['caption']:
            for s in a['caption']:
                jimg = {}
                jimg['imgid'] = img['id']
                jimg['raw'] = s
                jimg['sentid'] = img['id']+'_'+str(sentid)
                txt = []
                for sentence in s:
                    print(sentence) 
                    txt.append(list(jieba.cut(sentence)))
                jimg['tokens'] = txt
                jimg['sentids'] = []
                out_s.append(jimg)
        out_im['sentences']=out_s
        out_im['split'] = split
        out.append(out_im)
    out_json['images']=out
    out_json['dataset']='ai_challenger'
    output_file = os.path.join('./json_preprocess_data', 'coco_ai_challenger.json')
    json.dump(out_json, open(output_file, 'w'))
    return out

In [52]:
out_train_val = ai_challenger_preprocess()
# out_train_val = json.load(open(os.path.join('./json_preprocess_data', 'coco_ai_challenger.json'),'r'))

UnboundLocalError: local variable 'json' referenced before assignment